**For this part of the exam, you will create a Streamlit app that will allow users to predict the price of a home by inputting certain information about it.  It will include inputs for features of the home and produce a predicted price.**

Load in the provided ML model (part2-model-pipeline.joblib) and training data (part2-training-data.joblib) to determine which features were used and the range of values included in each feature. If you receive an error about no module named dill, run “!pip install dill” in your notebook before using joblib.load

In [1]:
pip install dill

Note: you may need to restart the kernel to use updated packages.


### Load in the filepaths.json file from the config folder.

In [2]:
# import joblib

# X_train, y_train = joblib.load('Models/part2-training-data.joblib')

In [3]:
# X_train.head()

In [4]:
# y_train.head()

In [5]:
# model_pipeline = joblib.load('Models/part2-model-pipeline.joblib')

In [6]:
# model_pipeline

In [7]:
import joblib, json
with open('config/filepaths.json') as f:
    FPATHS = json.load(f)
FPATHS

{'data': {'ml': {'train': 'data/part2-training-data.joblib',
   'test': 'data/part2-test-data.joblib'}},
 'models': {'linear_regression': 'models/part2-model-pipeline.joblib'}}

### Use the filepaths dictionary to load in the provided ML model and training data (X_train, y_train) to determine which features were used and the range of values included in each feature.

In [8]:
# # Define the filepath using the new dictionary structure
# # load train data 
# traindata = FPATHS['data']['ml']['train']
# [X_train,y_train] = joblib.load(traindata)

# # load test data 
# testdata = FPATHS['data']['ml']['test']
# [X_test,y_test] = joblib.load(testdata)

# # display Xtrain and Xtest

# X_train.head()


In [9]:
import streamlit as st
@st.cache_data
def load_Xy_data(fpath):
    train_path = fpath['data']['ml']['train']
    X_train, y_train =  joblib.load(train_path)
    test_path = fpath['data']['ml']['test']
    X_test, y_test = joblib.load(test_path)
    return X_train, y_train, X_test, y_test
 
@st.cache_resource
def load_model_ml(fpath):
    model_path = fpath['models']['linear_regression']
    linreg = joblib.load(model_path)
    return linreg

2024-01-18 13:29:34.107 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [10]:
X_train, y_train, X_test, y_test = load_Xy_data(FPATHS)

2024-01-18 13:29:34.203 
  command:

    streamlit run /opt/homebrew/Caskroom/miniforge/base/envs/dojo-env/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-01-18 13:29:34.204 No runtime found, using MemoryCacheStorageManager


In [11]:
X_test.head()

,bedrooms,bathrooms,sqft_living
id,,,
5416500660,4,2.50,2960
259801030,4,2.00,1610
7577700185,4,1.00,1440
1939000030,4,2.50,2540
7524950870,4,2.25,2110


In [12]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15905 entries, 2473372170 to 7806450190
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   bedrooms     15905 non-null  int64  
 1   bathrooms    15905 non-null  float64
 2   sqft_living  15905 non-null  int64  
dtypes: float64(1), int64(2)
memory usage: 497.0 KB


In [13]:
# # Now we can save our model using the filepath from the dictionary
# linreg_path = FPATHS['models']['linear_regression']
# linreg_path


# linreg_model = joblib.load(linreg_path)

In [14]:
linreg_model = load_model_ml(FPATHS)

In [15]:
linreg_model

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x1656d9ae0>),
                                                 ('num', StandardScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x1656da500>)],
                                   verbose_feature_names_out=False)),
                ('reg', LinearRegression())])

### Explore the features


In [16]:
X_train.describe()

,bedrooms,bathrooms,sqft_living
count,15905.000000,15905.000000,15905.000000
mean,3.349010,2.088086,2030.448287
std,0.925197,0.736655,837.357259
min,0.000000,0.000000,290.000000
25%,3.000000,1.500000,1420.000000
50%,3.000000,2.250000,1890.000000
75%,4.000000,2.500000,2500.000000
max,33.000000,7.500000,7480.000000


In [17]:
import streamlit as st
#bedrooms
bedrooms = st.sidebar.slider('Bedrooms',
                            min_value = X_train['bedrooms'].min(),
                            max_value = X_train['bedrooms'].max(),
                            step = 1, value = 3)


In [18]:
bedrooms

3

In [19]:
#bathrooms
bathrooms = st.sidebar.slider('Bathrooms',
                             min_value = X_train['bathrooms'].min(),
                             max_value = X_train['bathrooms'].max(),
                             step = .25, value = 2.5)

In [20]:
bathrooms

2.5

In [21]:
#sqft_living
sqft_living = st.sidebar.number_input('Sqft Living Area',
                                     min_value=290,
                                     max_value=X_train['sqft_living'].max(),
                                     step=150, value=2500)

In [22]:
# Define function to convert widget values to dataframe
import pandas as pd
def get_X_to_predict():
    X_to_predict = pd.DataFrame({'Bedroom': bedrooms,
                                 'Bathroom':bathrooms,
                                 'Living Area Sqft':sqft_living},
                             index=['House'])
    return X_to_predict

In [23]:
def get_prediction(model,X_to_predict):
    return  model.predict(X_to_predict)[0]

In [24]:
X_to_pred = get_X_to_predict()
X_to_pred

,Bedroom,Bathroom,Living Area Sqft
House,3,2.5,2500


In [25]:
type(X_to_pred)

pandas.core.frame.DataFrame

In [27]:
#print(get_prediction(linreg_model,X_to_pred))

In [ ]:
# #predict one value
# linreg_model.predict(X_to_pred)[0]

In [ ]:
X_train['bedrooms'].max()